In [1]:
import random

from deap import base
from deap import creator
from deap import tools

from PIL import Image, ImageChops
import numpy as np
import math, operator

C:\Users\Joe\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [ ]:
# set filename
# open image
# convert image to format ready to be compared to



In [ ]:
# create evaluate functions

# create mutate function - mutate floats, how to stay in range??

# create crossover function

In [140]:
from deap import base, creator, tools, algorithms
from random import randint, random, gauss
from PIL import Image, ImageDraw
from functools import partial
from math import sqrt
import numpy


PIC = Image.open('template.jpg')
SIZE = 100
NUMBER_OF_TRIANGLES = 30
POPULATION = 50
NGEN = 100
POLY = 3
VALUES_PER_GENE = 12

MUTATION_NUM_TO_RANDOM = 5

def computeSimilarity(allInds):
    numpyInds = numpy.array(allInds)
    groupedGenes = np.reshape(numpyInds.ravel(order='F'),(NUMBER_OF_TRIANGLES*VALUES_PER_GENE,POPULATION))
    numpy.ndarray.sort(groupedGenes,axis=1)
    
    diffs = numpy.apply_along_axis(np.diff,1,groupedGenes)
    
    multipliers = [i * (POPULATION-i) for i in range(POPULATION)]
    multipliers = multipliers[1:]
    
    scaledDiffs = numpy.apply_along_axis(lambda x: x*multipliers,1,diffs)
    averageDiffInGene = numpy.apply_along_axis(numpy.mean,1,diffs)
    
    return 1-numpy.mean(averageDiffInGene)


def gen_one_triangle():
    return [random() for i in range(VALUES_PER_GENE)]


def triangles_to_image(triangles):
    im = Image.new('RGB', (SIZE, SIZE), (255, 255, 255))
    for tri in triangles:
        mask = Image.new('RGBA', (SIZE, SIZE))
        draw = ImageDraw.Draw(mask)
        
        t = [int(x * SIZE) for x in tri[0:6]]
        triangle = ((t[0],t[1]),(t[2],t[3]),(t[4],t[5]))
        rgb = [int(x * 255) for x in tri[3:6]]
        fillAlpha = int(tri[5] * 30)

        fillColor = (rgb[0],rgb[1],rgb[2],fillAlpha)
        draw.polygon(triangle, fill=fillColor)
        im.paste(mask, mask=mask)
        del mask, draw
    return im


def evaluate(im1, t2):
    im2 = triangles_to_image(t2)

    h = ImageChops.difference(im1,im2).histogram()
    length = int(len(h)/3)

    r = h[length*0:length*1]
    g = h[length*1:length*2]
    b = h[length*2:length*3]

    h = [sum(x) for x in zip(r,g,b)]
    err = math.sqrt(sum(h*(i**2) for i, h in enumerate(h)) / float(SIZE * SIZE * 3)) / 256
    return 1 - err,



def mutate(triangles):
    for i in range(MUTATION_NUM_TO_RANDOM):
        triangleToMutate = triangles[randint(0,len(triangles)-1)]
        valueIndexToMutate = randint(0,len(triangleToMutate)-1)
        triangleToMutate[valueIndexToMutate] = random()
        
    return triangles,

creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("attr", gen_one_triangle)
toolbox.register("individual", tools.initRepeat,
                 creator.Individual, toolbox.attr, NUMBER_OF_TRIANGLES)
toolbox.register("population", tools.initRepeat,
                 list, toolbox.individual)

toolbox.register("evaluate", partial(evaluate, PIC))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)


def main():
    pop = toolbox.population(n=POPULATION)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_sim = tools.Statistics(lambda ind: ind)
    stats_fit.register("std", numpy.std)
    stats_fit.register("max", numpy.max)
    stats_fit.register("avg", numpy.mean)
    stats_fit.register("min", numpy.min)
    stats_sim.register("similarity",computeSimilarity)

    mstats = tools.MultiStatistics(fitness=stats_fit, similarity=stats_sim)

    log = ""
    try:
        pop, log = algorithms.eaSimple(
            pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=NGEN, stats=mstats,
            halloffame=hof, verbose=True)
    finally:
        open('stats.txt', 'w').write(str(log))
        open('result.txt', 'w').write(repr(hof[0]))
        triangles_to_image(hof[0]).save('result.png')
        
if __name__ == '__main__':
    main()

C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	                         fitness                         	similarity
   	      	---------------------------------------------------------	----------
gen	nevals	avg     	max     	min     	std      	similarity
0  	50    	0.445013	0.468524	0.425398	0.0101338	0.980416  
1  	28    	0.453028	0.470393	0.429203	0.0088413	0.98125   
2  	29    	0.459909	0.476915	0.437453	0.00841584	0.982222  
3  	30    	0.466124	0.491783	0.445334	0.00735939	0.984207  
4  	34    	0.470804	0.484836	0.451868	0.00697961	0.98547   
5  	23    	0.476942	0.491485	0.461233	0.00610293	0.986496  
6  	33    	0.480856	0.497398	0.465487	0.00731333	0.98714   
7  	29    	0.485331	0.497398	0.470925	0.00540062	0.989495  
8  	28    	0.490276	0.497724	0.477472	0.00488508	0.991073  
9  	20    	0.494302	0.50166 	0.485798	0.0030144 	0.991955  
10 	30    	0.49654 	0.505631	0.489166	0.0024834 	0.992497  
11 	36    	0.497772	0.505631	0.485461	0.00325669	0.99371   
12 	31    	0.498984	0.506003	0.489946	0.00283098	0.993824  
13 

In [65]:
import cProfile
cProfile.run("main()")

gen	nevals	std       	max     	avg     	min     
0  	75    	0.00913407	0.501332	0.470455	0.448969
1  	33    	0.00881925	0.501591	0.477101	0.458309
2  	41    	0.00888577	0.508514	0.484942	0.470293
3  	44    	0.00886675	0.507653	0.491934	0.476185
4  	33    	0.00636476	0.509895	0.499001	0.472964
5  	42    	0.00404629	0.51305 	0.503793	0.492986
6  	50    	0.00355603	0.51509 	0.506916	0.499301
7  	44    	0.00330504	0.517001	0.509833	0.501313
8  	45    	0.00290689	0.520649	0.513052	0.504608
9  	38    	0.00258472	0.523363	0.515206	0.507007
10 	38    	0.00290661	0.523754	0.517078	0.51055 
11 	36    	0.00253039	0.523754	0.519224	0.514816
12 	38    	0.00257395	0.529409	0.521477	0.512251
13 	52    	0.00141759	0.52711 	0.523093	0.517693
14 	42    	0.00165115	0.528553	0.524173	0.517957
15 	42    	0.00156004	0.530213	0.525474	0.521445
16 	38    	0.00129205	0.5304  	0.526495	0.523139
17 	28    	0.00166535	0.5304  	0.527711	0.521595
18 	33    	0.00122165	0.532242	0.528972	0.525634
19 	48    	0.0009139